In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# For sentiment
from transformers import pipeline

/Users/npop/Code/Projects/ML7641_Project/DLenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
comments = pd.read_csv('/Users/npop/Code/Projects/ML7641_Project/Official_Datasets/stage_2/reddit_comments_sentiment.csv')
print(comments.isna().sum())
comments.dropna(inplace=True)

/var/folders/jw/9mhjkj2530q0dcv2dbdg1zmm0000gn/T/ipykernel_7944/1014335402.py:1: DtypeWarning: Columns (2,10) have mixed types. Specify dtype option on import or set low_memory=False.
  comments = pd.read_csv('/Users/npop/Code/Projects/ML7641_Project/Official_Datasets/stage_2/reddit_comments_sentiment.csv')


body                   0
post_date              9
upvotes                9
parent_id              9
top_level_id           9
post_title             9
post_id                9
movie_title            9
movie_release_date     9
movie_actors           9
valid_post_date       10
dtype: int64


In [2]:
#base_sentiment_pipeline = pipeline('sentiment-analysis')
model_path = "cardiffnlp/twitter-roberta-base-sentiment-latest"
sentiment_pipeline = pipeline('sentiment-analysis', model=model_path, tokenizer=model_path)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
sentiments = []
for i, comment in enumerate(comments['body']):
    try:
        sentiment = sentiment_pipeline(comment)
    except:
        sentiment = 'error'
        print(f'There was an error for comment {i}: {comment}')
    sentiments.append(sentiment)

There was an error for comment 1283: Some Thoughts on the Science in Gravity

Several things about this trailer are hard to explain from a space science point of view Someone brought up several criticisms in comments on Wired and I responded basically as below Id like to reproduce that discussion here the original can be read at httpwwwwiredcomunderwire201305firstgravitymovietrailer

Curiosities 
Why are there explosions Things in space arent made of dynamite 
Why is the ISS shown burning up in the atmosphere Even if you ripped ISS to pieces it would take months for them to reenter
Wouldnt it be really hard to make something heavy like the Space Shuttle Orbiter spin around like that Youd have to hit it with a bus
Why is there sound in space

Trailers like this are a great opportunity to discuss how science and engineering are presented in film so its good people are getting that discussion started Of course the writers and director should be taken to task for anything that proves to be

In [9]:
comments['sentiment'] = sentiments

In [10]:
comments.head()

,body,post_date,upvotes,parent_id,top_level_id,post_title,post_id,movie_title,movie_release_date,movie_actors,valid_post_date,sentiment
0,Damn even Scorsese pirated Yeezus,2013-06-17,697.0,Top Level,cakjh4s,Scorsese's 'The Wolf Of Wall Street' Trailer D...,1ghzr9,The Wolf of Wall Street,2013-12-25,"Leonardo DiCaprio, Jonah Hill, Margot Robbie",True,"[{'label': 'negative', 'score': 0.483877182006..."
1,Leo loves it though httpiimgurcomcPTIWougif,2013-06-17,168.0,cakjh4s,cakjh4s,Scorsese's 'The Wolf Of Wall Street' Trailer D...,1ghzr9,The Wolf of Wall Street,2013-12-25,"Leonardo DiCaprio, Jonah Hill, Margot Robbie",True,"[{'label': 'positive', 'score': 0.915180742740..."
2,Not as much as Ethan SupleehttpiimgurcomxNcCDs...,2013-06-17,71.0,cakpae0,cakjh4s,Scorsese's 'The Wolf Of Wall Street' Trailer D...,1ghzr9,The Wolf of Wall Street,2013-12-25,"Leonardo DiCaprio, Jonah Hill, Margot Robbie",True,"[{'label': 'negative', 'score': 0.502026021480..."
3,Please put white people over this,2013-06-17,25.0,cakpae0,cakjh4s,Scorsese's 'The Wolf Of Wall Street' Trailer D...,1ghzr9,The Wolf of Wall Street,2013-12-25,"Leonardo DiCaprio, Jonah Hill, Margot Robbie",True,"[{'label': 'negative', 'score': 0.677761197090..."
4,close up of dat facehttpiimgurcombcWbPTFgif,2013-06-17,3.0,cakpae0,cakjh4s,Scorsese's 'The Wolf Of Wall Street' Trailer D...,1ghzr9,The Wolf of Wall Street,2013-12-25,"Leonardo DiCaprio, Jonah Hill, Margot Robbie",True,"[{'label': 'neutral', 'score': 0.7312556505203..."


# Sentiment Analysis for YouTube comments

In [3]:
youtube_comments = pd.read_csv("/Users/npop/Code/Projects/ML7641_Project/Official_Datasets/stage_2/youtube_comments_valid.csv")

In [5]:
youtube_comments.drop_duplicates(subset=['comment_text', 'comment_date'], inplace=True)

In [8]:
youtube_comments['comment_date'] = pd.to_datetime(youtube_comments['comment_date'])
youtube_comments['movie_release_date'] = pd.to_datetime(youtube_comments['movie_release_date'])

In [21]:
def clean_comment(comment):
    import re
    url_pattern = r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)'
    comment = re.sub(url_pattern, '', comment)
    pattern = r'[^\w\s\d]'
    comment = re.sub(pattern, '', comment)
    pattern = r'\xa0|\n'
    comment = re.sub(pattern, ' ', comment)
    double_space = r'\s\s'
    comment = re.sub(double_space, ' ', comment)
    return comment.lower()

In [22]:
youtube_comments['comment_text_clean'] = youtube_comments['comment_text'].apply(clean_comment)

In [23]:
youtube_comments.loc[0, 'comment_text_clean']

'its about time leo gets an oscar already  and he also has the most awesome birthday date of the year '

In [25]:
youtube_sentiments = []
for i, comment in enumerate(youtube_comments['comment_text_clean']):
    try:
        sentiment = sentiment_pipeline(comment)
    except:
        sentiment = 'error'
        print(f'There was an error for comment {i} {comment[:50]}')
    youtube_sentiments.append(sentiment)

There was an error for comment 9453 the battle for the brain by dante kohrs mar 2014 m
There was an error for comment 12919 it may be a decent effort but not really convinced
There was an error for comment 17431 peter jackson is best living man on earth before y
There was an error for comment 18933  heres your history lesson persians came twice to 
There was an error for comment 19032  типичные американцы вот классика жанра прям  доро
There was an error for comment 20762 people in the comments are so ignorant and im star
There was an error for comment 27561  then maziar bahari came for me it was around noon
There was an error for comment 28625 silent night 2021 i got free from here whatch now 
There was an error for comment 28626 single all the way 2021 i got free from here whatc
There was an error for comment 28627 single all the way 2021 i got free from here whatc
There was an error for comment 28628 silent night 2021  i got free from here whatch now
There was an error for comment 28

In [26]:
youtube_comments['sentiment'] = youtube_sentiments

In [28]:
youtube_comments.to_csv('/Users/npop/Code/Projects/ML7641_Project/Official_Datasets/stage_2/youtube_comments_sentiment_scored.csv', index=False)

# Sentiment Analysis (Aggregations)

In [65]:
reddit_sentiment_comments = pd.read_csv("/Users/npop/Code/Projects/ML7641_Project/Official_Datasets/stage_2/reddit_comments_sentiment_scored.csv")
youtube_sentiment_comments = pd.read_csv("/Users/npop/Code/Projects/ML7641_Project/Official_Datasets/stage_2/youtube_comments_sentiment_scored.csv")

In [101]:
reddit_sentiment_comments.head(1)
#youtube_sentiment_comments.head(1)

,body,post_date,upvotes,parent_id,top_level_id,post_title,post_id,movie_title,movie_release_date,movie_actors,valid_post_date,sentiment,sentiment_label,sentiment_conf,sentiment_score
0,Damn even Scorsese pirated Yeezus,2013-06-17,697.0,Top Level,cakjh4s,Scorsese's 'The Wolf Of Wall Street' Trailer D...,1ghzr9,The Wolf of Wall Street,2013-12-25,"Leonardo DiCaprio, Jonah Hill, Margot Robbie",True,"[{'label': 'negative', 'score': 0.483877182006...",negative,0.483877,-1.0


In [72]:
youtube_sentiment_comments.dropna(inplace=True)

In [91]:
def extract_sentiment_label(sentiment_text):
    #sentiment_text = sentiment_text[0]
    if 'positive' in sentiment_text:
        return "positive"
    elif 'negative' in sentiment_text:
        return "negative"
    elif 'neutral' in sentiment_text:
        return "neutral"
    else:
        return "error"
        

def extract_sentiment_score(sentiment_text):
    import re
    #sentiment_text = sentiment_text[0]
    #if type(sentiment_text) == str:
        #return np.nan
    digits = re.findall(r'0\.\d+', sentiment_text)
    if digits:
        return digits[0]
    else:
        return np.nan


youtube_sentiment_comments['sentiment_label'] = youtube_sentiment_comments['sentiment'].apply(extract_sentiment_label)
youtube_sentiment_comments['sentiment_conf'] = youtube_sentiment_comments['sentiment'].apply(extract_sentiment_score)

In [93]:
youtube_sentiment_comments.to_csv('/Users/npop/Code/Projects/ML7641_Project/Official_Datasets/stage_2/youtube_comments_sentiment_scored.csv', index=False)

In [96]:
reddit_sentiment_comments.rename(columns={'sentiment_score': 'sentiment_conf'}, inplace=True)

In [97]:
def convert_sentiment_2score(label:str):
    """Converts sentiment label to -1, 0, or 1

    Args:
        label (str): sentiment label

    Returns:
        int: integer corresonding with sentiment score
    """
    if label == 'negative':
        return -1
    elif label == 'neutral':
        return 0
    elif label == 'positive':
        return 1
    else:
        return np.nan

In [98]:
youtube_sentiment_comments['sentiment_score'] = youtube_sentiment_comments['sentiment_label'].apply(convert_sentiment_2score)
reddit_sentiment_comments['sentiment_score'] = reddit_sentiment_comments['sentiment_label'].apply(convert_sentiment_2score)

In [102]:
# convert upvotes back to float dtype
youtube_sentiment_comments['comment_likes'] = youtube_sentiment_comments['comment_likes'].astype(float)
reddit_sentiment_comments['upvotes'] = reddit_sentiment_comments['upvotes'].astype(float)

In [106]:
# scaling comments by log and sqrt
def scale_sentiment(row:pd.Series, func='sqrt'):
    upvotes = row['upvotes']
    upvotes_abs = np.abs(upvotes)
    sentiment = row['sentiment_score']
    # handle potential nan values
    if np.isnan(sentiment):
        return np.nan
    # conditionals to check for type of scaling
    if func == 'log':
        scaled_sentiment = sentiment * np.log(upvotes_abs + 1)
    elif func == 'sqrt':
        scaled_sentiment = sentiment * np.sqrt(upvotes_abs + 1)
    # if negative upvotes, then the sentiment should be reversed
    if upvotes < 0.0:
        return scaled_sentiment * -1
    else:
        return scaled_sentiment

In [105]:
youtube_sentiment_comments['scaled_sentiment_sqrt'] = youtube_sentiment_comments.apply(scale_sentiment, axis=1)

In [107]:
reddit_sentiment_comments['scaled_sentiment_sqrt'] = reddit_sentiment_comments.apply(scale_sentiment, axis=1)

## Aggregating Stats for Each Entry

In [111]:
def generate_movie_stats(movie_comments_group, platform) -> list[pd.DataFrame]:
    # combined upvotes from all comments
    if platform == 'youtube':
        vote_name = 'comment_likes'
    else:
        vote_name = 'upvotes'
    
    movie_stats_df = movie_comments_group[vote_name].sum().reset_index(drop=False)
    # number positive & negative comments
    value_counts = movie_comments_group['sentiment_label'].value_counts().reset_index(name='label_count')
    value_counts = value_counts.pivot(index='movie_title', columns='sentiment_label', values='label_count').reset_index()
    value_counts.rename(columns={'negative': 'neg_comments', 'neutral': 'neut_comments', 'positive': 'pos_comments', 'error': 'error_comments'}, inplace=True)
    value_counts.columns.name = None
    # positive/negative comment ratio
    value_counts['pos_neg_ratio'] = value_counts['pos_comments'] / value_counts['neg_comments']
    # scaled sentiment 
    scaled_sqrt = movie_comments_group.agg({'scaled_sentiment_sqrt': ['mean', 'sum']}).reset_index(drop=False)
    scaled_sqrt.columns = ['movie_title', 'mean_scaled_sent_sqrt', 'sum_scaled_sent_sqrt']
    return [movie_stats_df, value_counts, scaled_sqrt]

In [113]:
yt_comments_group = youtube_sentiment_comments.groupby('movie_title')
reddit_comments_group = reddit_sentiment_comments.groupby('movie_title')
yt_comm_stats = generate_movie_stats(yt_comments_group, 'youtube')
red_comm_stats = generate_movie_stats(reddit_comments_group, 'reddit')

In [115]:
reddit_comment_stats = pd.merge(red_comm_stats[0], red_comm_stats[1], on='movie_title', how='inner')
reddit_comment_stats = pd.merge(reddit_comment_stats, red_comm_stats[2], on='movie_title', how='inner')

yt_comment_stats = pd.merge(yt_comm_stats[0], yt_comm_stats[1], on='movie_title', how='inner')
yt_comment_stats = pd.merge(yt_comment_stats, yt_comm_stats[2], on='movie_title', how='inner')

In [117]:
reddit_comment_stats.fillna(0, inplace=True)
yt_comment_stats.fillna(0, inplace=True)

In [118]:
reddit_comment_stats.to_csv('/Users/npop/Code/Projects/ML7641_Project/Official_Datasets/stage_2/reddit_comments_stats.csv', index=False)
yt_comment_stats.to_csv('/Users/npop/Code/Projects/ML7641_Project/Official_Datasets/stage_2/youtube_comments_stats.csv', index=False)